In [1]:
PATH = '/Users/siddhantthakur/pizza req/'

import json
import pandas as pd

import matplotlib.pyplot as plt

import nltk
from nltk.tokenize import regexp_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import re

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import warnings
warnings.filterwarnings("ignore")

In [2]:
# certificate verification fail

import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

In [3]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/siddhantthakur/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/siddhantthakur/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/siddhantthakur/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [4]:
with open(PATH + 'pizza_request_dataset/pizza_request_dataset.json', 'r') as f:
    data = json.load(f)

In [5]:
len(data)

5671

In [6]:
sorted(data[0].keys())

['giver_username_if_known',
 'in_test_set',
 'number_of_downvotes_of_request_at_retrieval',
 'number_of_upvotes_of_request_at_retrieval',
 'post_was_edited',
 'request_id',
 'request_number_of_comments_at_retrieval',
 'request_text',
 'request_text_edit_aware',
 'request_title',
 'requester_account_age_in_days_at_request',
 'requester_account_age_in_days_at_retrieval',
 'requester_days_since_first_post_on_raop_at_request',
 'requester_days_since_first_post_on_raop_at_retrieval',
 'requester_number_of_comments_at_request',
 'requester_number_of_comments_at_retrieval',
 'requester_number_of_comments_in_raop_at_request',
 'requester_number_of_comments_in_raop_at_retrieval',
 'requester_number_of_posts_at_request',
 'requester_number_of_posts_at_retrieval',
 'requester_number_of_posts_on_raop_at_request',
 'requester_number_of_posts_on_raop_at_retrieval',
 'requester_number_of_subreddits_at_request',
 'requester_received_pizza',
 'requester_subreddits_at_request',
 'requester_upvotes_minus

In [7]:
success = [d['requester_received_pizza'] for d in data]
print("Average Success Rate",str(sum(success)/len(success)*100))

Average Success Rate 24.634103332745546


In [8]:
for d in data:
    d['upvotes_minus_downvotes'] = d['number_of_upvotes_of_request_at_retrieval'] - d['number_of_downvotes_of_request_at_retrieval']

for d in data:
    d['final_request_text'] = d['request_title'] + " " + d['request_text_edit_aware']

In [9]:
data[0]

{'giver_username_if_known': 'N/A',
 'in_test_set': False,
 'number_of_downvotes_of_request_at_retrieval': 2,
 'number_of_upvotes_of_request_at_retrieval': 6,
 'post_was_edited': False,
 'request_id': 't3_w5491',
 'request_number_of_comments_at_retrieval': 7,
 'request_text': "I'm not in College, or a starving artist or anything like that. I've just been a bit unlucky lately. I'm a 36 year old single guy with a job. But rent, and other bills killed me this month. I thought I had enough funds in my account to at least keep me set with Mr. Noodles, I forgot about my monthly banking fee.\n\nI had a small bag of chips Wednesday afternoon, and I get paid Monday, so I'll be fine then.. It's just really painful at this point and food is something I'm constantly thinking about.\n\nI've got a few bucks to get on the bus to work on Saturday, so I can't really use that.\n\nI'm really embarrassed to even be asking like this and I'm not sure how it works, so please be patient with me.\n\nI guess tha

In [10]:
df = pd.json_normalize(data)
df['requester_received_pizza'] = df['requester_received_pizza'].astype(int)
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,giver_username_if_known,in_test_set,number_of_downvotes_of_request_at_retrieval,number_of_upvotes_of_request_at_retrieval,post_was_edited,request_id,request_number_of_comments_at_retrieval,request_text,request_text_edit_aware,request_title,...,requester_upvotes_minus_downvotes_at_request,requester_upvotes_minus_downvotes_at_retrieval,requester_upvotes_plus_downvotes_at_request,requester_upvotes_plus_downvotes_at_retrieval,requester_user_flair,requester_username,unix_timestamp_of_request,unix_timestamp_of_request_utc,upvotes_minus_downvotes,final_request_text
0,N/A,False,2,7,False,t3_jyf1u,13,"I'm just hanging out with my sibling, bored as...","I'm just hanging out with my sibling, bored as...",[REQUEST] In southern New Hampshire and broke ...,...,450,7661,2390,14697,None,AttractiveMeat,1.314661e+09,1.314657e+09,5,[REQUEST] In southern New Hampshire and broke ...
1,N/A,False,7,8,False,t3_1bydsc,1,I'm in between jobs right now and am living in...,I'm in between jobs right now and am living in...,[Request] haven't eaten in three days; pizza w...,...,565,564,1669,1688,None,thablackphoenix,1.365470e+09,1.365466e+09,1,[Request] haven't eaten in three days; pizza w...
2,N/A,False,2,4,False,t3_146rcc,0,"Starving student, just making it by, have a fe...","Starving student, just making it by, have a fe...","[request] AZ, US, hardworking engineering stud...",...,222,238,506,538,None,jlmango,1.354514e+09,1.354514e+09,2,"[request] AZ, US, hardworking engineering stud..."
3,N/A,False,2,2,False,t3_1l39ub,0,...I've spent my weekly allowance on other nec...,...I've spent my weekly allowance on other nec...,"[Request] [UK] Basically, I recently moved int...",...,9168,11410,38596,43364,None,felixthemaster1,1.377486e+09,1.377482e+09,0,"[Request] [UK] Basically, I recently moved int..."
4,N/A,False,1,3,False,t3_t3nc3,4,with pizza. \nehh??\n\n\nWatching Star Trek wi...,with pizza. \nehh??\n\n\nWatching Star Trek wi...,"[request] broke, hungry, and in love..",...,106,138,210,250,shroom,omyhotlibertineson,1.335987e+09,1.335983e+09,2,"[request] broke, hungry, and in love.. with pi..."


In [11]:
df['requester_received_pizza'].isna().sum()

0

In [12]:
# text preprocessing
stopwords_eng = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

def process_text(text):
    text = text.replace("\n"," ").replace("\r"," ")
    text = re.sub(r'“', " '' ", text)
    text = re.sub(r'”', " '' ", text)
    text = re.sub(r'"', " '' ", text)
    text = re.sub(r'(\S)(\'\')', r'\1 \2 ', text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    
    punc_list = '!"#$%()*+,-./:;<=>?@^_{|}~[]'
    t = str.maketrans(dict.fromkeys(punc_list," "))
    text = text.translate(t)
    
    t = str.maketrans(dict.fromkeys("'`",""))
    text = text.translate(t)
    
    text = text.lower()
    
    tokens = regexp_tokenize(text,pattern='\s+',gaps=True)
    cleaned_tokens = []
    
    for t in tokens:
        if t not in stopwords_eng:
            l = lemmatizer.lemmatize(t)
            cleaned_tokens.append(l)
    
    return cleaned_tokens

In [13]:
process_text(df['final_request_text'][0])

['request',
 'southern',
 'new',
 'hampshire',
 'broke',
 'hell',
 'hanging',
 'sibling',
 'bored',
 'hell',
 'nothing',
 'interesting',
 'god',
 'would',
 'suck',
 'thousand',
 'dick',
 'buffalo',
 'chicken',
 'pizza',
 'anybody',
 'reply',
 'heaven',
 'penis',
 'take']

In [14]:
text_data = df[['final_request_text','requester_received_pizza']]
text_data['tokenized_text'] = text_data.apply(lambda r: process_text(r['final_request_text']), axis = 1)
text_data.head(10)

,final_request_text,requester_received_pizza,tokenized_text
0,[REQUEST] In southern New Hampshire and broke ...,0,"[request, southern, new, hampshire, broke, hel..."
1,[Request] haven't eaten in three days; pizza w...,0,"[request, eaten, three, day, pizza, would, gre..."
2,"[request] AZ, US, hardworking engineering stud...",0,"[request, az, u, hardworking, engineering, stu..."
3,"[Request] [UK] Basically, I recently moved int...",0,"[request, uk, basically, recently, moved, new,..."
4,"[request] broke, hungry, and in love.. with pi...",1,"[request, broke, hungry, love, pizza, ehh, wat..."
5,[REQUEST] Following my dreams. Chose my band o...,0,"[request, following, dream, chose, band, job, ..."
6,"[Request] Astoria, Queens, NY could totally us...",0,"[request, astoria, queen, ny, could, totally, ..."
7,"[Request] Ontario Canada, Hanging out with my ...",0,"[request, ontario, canada, hanging, cousin, ge..."
8,[REQUEST] UK - Hungry couple with no money (ye...,1,"[request, uk, hungry, couple, money, yeah, kno..."
9,Hope this is the right place [Request] NC Hey ...,1,"[hope, right, place, request, nc, hey, guy, ho..."


In [15]:
count_vec = CountVectorizer(analyzer = process_text)
count_df = count_vec.fit_transform(text_data['final_request_text']).toarray()
count_labels = text_data['requester_received_pizza']

# count_df vs count_labels [different models]

In [16]:
tfidf_vec = TfidfVectorizer(analyzer = process_text)
tfidf_df = tfidf_vec.fit_transform(text_data['final_request_text']).toarray()
tfidf_labels = text_data['requester_received_pizza']

# tfidf_df vs tfidf_labels [different models]